<a href="https://colab.research.google.com/github/kgautam2103/Zoom_Speech2text_GCP_public/blob/main/Zoom_Speech2text_GCP_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("mnt",force_remount=True)
%run '/content/mnt/MyDrive/Colab Notebooks/zoom_speech2text_utility_functions.ipynb'
!cp /content/mnt/MyDrive/bigdata-poc-281913-c1c79233a51d.json /content/adc.json


Mounted at mnt


In [ ]:
!pip3 install pydub
!pip3 install google-cloud
!pip3 install google-cloud-speech


In [ ]:
import requests
import os
import wave
from pydub import AudioSegment
from google.cloud import storage
from urllib.parse import urlencode
from google.cloud import speech_v1p1beta1 as speech
from google.colab import files


meeting_id = input("enter the meeting id like 9181716151 : ")
recording_date = input("enter the recording date like 2021-07-25 : ")
base_url = 'https://api.zoom.us/v2/'
past_meetings = base_url+'/past_meetings/'
get_meeting_url = past_meetings+meeting_id+'/instances'
cwd = os.getcwd()
print(cwd)

#put the auth token for the zoom account
auth_token = ''



authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

meeting_id_response = requests.get(get_meeting_url,headers=auth_headers)

if meeting_id_response.status_code == 200:

  for x in meeting_id_response.json()['meetings']:
    if recording_date in str(x['start_time']):
      print(x)
   

enter the meeting id like 9181716151 : 9181716151
enter the recording date like 2021-07-25 : 2021-08-06
/content
{'uuid': 'A5JzJ1rqR6CMMi9KpLF0Hg==', 'start_time': '2021-08-06T10:04:22Z'}
{'uuid': 'EcQCnXqRRa2tt9MF+PFHRg==', 'start_time': '2021-08-06T16:44:37Z'}
{'uuid': 'IPDUNB7cSU6nXpgT/YINnw==', 'start_time': '2021-08-06T16:02:47Z'}
{'uuid': 'p7Kl684GRMuXrYjH/8VAKA==', 'start_time': '2021-08-06T00:31:55Z'}


In [ ]:
#based on the above result, please enter the meeting uuid
meeting_uuid = input("enter the meeting uuid like wHc5q3qBT2+RfFdg8Gt9dg== : ")
project_id='bigdata-poc-281913'
gcs_bucket_name='speech-to-audio'
key='/content/adc.json'

double_encoded_meeting_uuid = double_encode_uuid(meeting_uuid)

recording_url='meetings/'+double_encoded_meeting_uuid+'/recordings'
final_recording_url=base_url+recording_url

recording_response = requests.get(final_recording_url,headers=auth_headers)

recording_download_url,start_ts = get_recording_download_url(recording_response.json())
download_response = requests.get(recording_download_url)

download_file_name=start_ts+".m4a"
wav_file_name = start_ts+".wav"
transcript_file_name = cwd+"/"+start_ts+".txt"

open(download_file_name,'wb').write(download_response.content)
print("m4a written to local")

input_filepath = cwd+"/"+download_file_name
wav_filepath = m4a_to_wav(input_filepath)
print("m4a converted to wav")

os.environ.setdefault("GCLOUD_PROJECT", project_id)
os.environ.setdefault("GOOGLE_APPLICATION_CREDENTIALS", key)
upload_blob(gcs_bucket_name,wav_filepath,wav_file_name)
print("file uploaded to GCS")

transcript = google_transcribe(cwd,gcs_bucket_name,wav_file_name)
write_transcripts(transcript_file_name,transcript)
print("transcript written to the file")
files.download(transcript_file_name)

enter the meeting uuid like wHc5q3qBT2+RfFdg8Gt9dg== : IPDUNB7cSU6nXpgT/YINnw==
m4a written to local
m4a converted to wav
file uploaded to GCS
32000
1
transcript written to the file


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>